# 2B Модель

In [ ]:
import chromadb
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma

# 🔹 Загружаем модель 
model_name = "google/gemma-2b-it" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# 🔹 Создаем пайплайн для инференса
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Подгружаем 8B модель

In [1]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quant_config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             quantization_config=quant_config,
                                             device_map="auto")

# Создаем пайплайн для модели
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
import json

def extract_location(query: str):
    """Извлекает страны и города с помощью LLM в формате few-shot."""

    prompt = f"""Извлеки названия стран и городов из запроса. Ответ строго в формате JSON.

    Пример 1:
    Текст: "Какие достопримечательности есть в Москве и Париже?"
    Ответ:
    {{
      "cities": ["Москва", "Париж"],
      "countries": ["Россия", "Франция"]
    }}
    
    Пример 2:
    Текст: "Что посмотреть в Берлине, Лондоне и Италии?"
    Ответ:
    {{
      "cities": ["Берлин", "Лондон"],
      "countries": ["Германия", "Великобритания", "Италия"]
    }}
    
    Пример 3:
    Текст: "{query}"
    Ответ:
    """

    response = llm_pipeline(prompt, max_new_tokens=100, do_sample=False)
    response_text = response[0]["generated_text"]

    try:
        # Разделим на части по слову 'Ответ' и возьмём сгенерированный ответ
        parts = response_text.split("Ответ:")
        answer_part = parts[3] 

        # Найдём первую подстроку от "{" до "}" 
        json_start = answer_part.find("{")
        json_end = answer_part.find("}") + 2

        if json_start == -1 or json_end == -1:
            raise ValueError("Не удалось найти JSON-блок")

        json_str = answer_part[json_start:json_end]
        result = json.loads(json_str)

    except Exception as e:
        print(f"Ошибка извлечения: {e}")
        result = {"cities": [], "countries": []}

    return result

#user_query = "Что можно посмотреть в Казани и в Германии?"
#print(extract_location(user_query))

# Тестирование ретривера


```
Предполагается, что модель сможет вычленять названия стран и городов из вопросов пользователей, соответственно мы можем оценить ретривер путем оценки соответствия извлеченных сущностей запросу. Тк в дальнейшем мы будем фильтровать чанки по мета информации (Страна, город) в контекст модели не будет попадать нерелевантная информация
```

## Тестовые запросы и правильные ответы

In [47]:
test_queries = [
    "Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?",
    "Что лучше попробовать из еды в Париже и Лионе?",
    "Какие музеи есть в Берлине и Мюнхене?",
    "Где можно покататься на гондолах в Италии?",
    "Какой пляжный отдых можно найти во Флоренции или Венеции?",
    "Какой общественный транспорт лучше использовать в Риме?",
    "Какова история Красной площади в Москве?",
    "Какие замки можно посетить во Франции?",
    "Какие современные здания есть в Берлине?",
    "Что стоит посетить в Казани туристу впервые?"
]

expected_entities = [
    {"cities": ["Москва", "Санкт-Петербург"], "countries": ["Россия"]},
    {"cities": ["Париж", "Лион"], "countries": ["Франция"]},
    {"cities": ["Берлин", "Мюнхен"], "countries": ["Германия"]},
    {"cities": [], "countries": ["Италия"]},  # Вопрос без конкретных городов
    {"cities": ["Флоренция", "Венеция"], "countries": ["Италия"]},
    {"cities": ["Рим"], "countries": ["Италия"]},
    {"cities": ["Москва"], "countries": ["Россия"]},
    {"cities": [], "countries": ["Франция"]},
    {"cities": ["Берлин"], "countries": ["Германия"]},
    {"cities": ["Казань"], "countries": ["Россия"]}
]

## Метрики оценки извлечения сущностей

### 1. Full Match Accuracy 

Эта метрика показывает, насколько точно модель извлекает все нужные сущности (города и страны).

**Как считается**:

Для каждого запроса проверяется, совпадают ли оба списка — и стран, и городов — в точности с ожидаемыми значениями:

Если совпадает — считаем 1 балл.
Если хотя бы одна лишняя или пропущенная сущность — 0 баллов.

Затем подсчитывается доля таких полностью верных предсказаний среди всех запросов.


### 2. IoU (Intersection over Union)

Метрика оценивает пересечение между предсказанными и ожидаемыми сущностями — то есть насколько сильно они совпадают, даже если не идеально.

**Как считается**:

Сначала отдельно для городов и отдельно для стран:

Считается, сколько сущностей было предсказано верно.
Сравнивается это с общим числом всех уникальных сущностей в предсказании и эталоне (то есть объединением).
Полученные значения объединяются в одно среднее число.

Затем IoU усредняется по всем запросам


In [3]:
from typing import List, Dict

def evaluate_retriever(predicted_entities: List[Dict],
                       queries: List[str],
                       expected: List[Dict]):
    """Оценивает точность выделения стран и городов альтернативными метриками."""

    correct_full_predictions = 0
    total_iou = 0

    for i, (pred, true) in enumerate(zip(predicted_entities, expected)):
        # Удаляем дубликаты
        pred_cities = set(pred.get("cities", []))
        pred_countries = set(pred.get("countries", []))

        true_cities = set(true.get("cities", []))
        true_countries = set(true.get("countries", []))

        # Проверка на точное совпадение
        if pred_cities == true_cities and pred_countries == true_countries:
            correct_full_predictions += 1

        # IoU по городам
        union_cities = pred_cities | true_cities
        inter_cities = pred_cities & true_cities
        cities_iou = len(inter_cities) / len(union_cities) if union_cities else 1.0

        # IoU по странам
        union_countries = pred_countries | true_countries
        inter_countries = pred_countries & true_countries
        countries_iou = len(inter_countries) / len(union_countries) if union_countries else 1.0

        avg_iou = (cities_iou + countries_iou) / 2
        total_iou += avg_iou

        print(f"\nВопрос {i+1}: {queries[i]}")
        print(f"Ожидалось: {true}")
        print(f"Предсказано: {pred}")
        print(f"IoU: {avg_iou:.2f}")
        print(f"Точное совпадение: {int(pred_cities == true_cities and pred_countries == true_countries)}")

    print("\n**Общая оценка**")
    print(f"Полностью верных предсказаний: {correct_full_predictions / len(queries):.2f}")
    print(f"Средний IoU: {total_iou / len(queries):.2f}")


In [49]:
# Инференсим модель и сохраняем ответы
predicted_entities = [extract_location(query) for query in test_queries]

In [50]:
predicted_entities

[{'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']},
 {'cities': ['Париж', 'Лион'], 'countries': ['Франция']},
 {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']},
 {'cities': ['Италия'], 'countries': ['Италия']},
 {'cities': ['Флоренция', 'Венеция'], 'countries': ['Италия']},
 {'cities': ['Рим'], 'countries': ['Италия']},
 {'cities': ['Москва'], 'countries': ['Россия']},
 {'cities': [], 'countries': ['Франция']},
 {'cities': ['Берлин'], 'countries': ['Германия']},
 {'cities': ['Казань'], 'countries': ['Россия']}]

In [63]:
evaluate_retriever(predicted_entities, test_queries, expected_entities)


Вопрос 1: Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?
Ожидалось: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия']}
Предсказано: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 2: Что лучше попробовать из еды в Париже и Лионе?
Ожидалось: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
Предсказано: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
IoU: 1.00
Точное совпадение: 1

Вопрос 3: Какие музеи есть в Берлине и Мюнхене?
Ожидалось: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']}
Предсказано: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 4: Где можно покататься на гондолах в Италии?
Ожидалось: {'cities': [], 'countries': ['Италия']}
Предсказано: {'cities': ['Италия'], 'countries': ['Италия']}
IoU: 0.50
Точное совпадение: 0

Вопрос 5: Какой пляжный отдых можно найти во Флоренции или 

# Тестирование генерации

## Загрузка данных

In [4]:
from chromadb import Client
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Загружаем эмбеддер FRIDA
embedding = HuggingFaceEmbeddings(
    model_name="ai-forever/FRIDA"
)

# Загружаем Chroma DB 
db = Chroma(
    embedding_function=embedding,
    persist_directory="chroma_storage"
)

C:\Users\kiril\AppData\Local\Temp\ipykernel_17252\1559281647.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_17252\1559281647.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [5]:
def retrieve_documents(query: str, top_k: int):
    """
    Получение релевантных документов с фильтрацией по городам, затем fallback на страну.
    """
    
    # Извлекаем страны и города
    location = extract_location(query)
    cities = location.get("cities", [])
    countries = location.get("countries", [])

    docs = []

    # Сначала пробуем искать по городам 
    if cities:
        city_filter = {"city": {"$in": cities}}

        try:
            retriever = db.as_retriever(search_kwargs={"filter": city_filter, "k": top_k})
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Ошибка при фильтрации по городам: {e}")

    # Если по городам ничего не нашли, пробуем по странам
    if not docs and countries:
        country_filter = {"country": {"$in": countries}}

        try:
            retriever = db.as_retriever(search_kwargs={"filter": country_filter, "k": top_k})
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Ошибка при фильтрации по странам: {e}")

    # Если ничего не найдено 
    if not docs:
        return "Информации в базе данных нет по указанным странам или городам."

    # Убираем префикс search_document:
    context = "\n\n".join(
        doc.page_content.replace("search_document:", "").strip() for doc in docs
    )

    return context

In [6]:
def generate_answer(query: str, context: str, max_new_tokens: int = 300) -> str:
    """
    Генерирует ответ с помощью LLM на основе контекста и вопроса пользователя.
    
    :param query: Вопрос пользователя
    :param context: Содержимое релевантных документов (из базы)
    :param max_new_tokens: Максимальное число новых токенов в ответе
    :return: Ответ модели
    """

    prompt = f"""
             Ты — интеллектуальный туристический помощник.
             Используя предоставленную информацию, ответь на вопрос пользователя.
                
             Контекст:
             {context}
                
             Вопрос: {query}
             Ответ:
             """

    response = llm_pipeline(prompt, max_new_tokens=max_new_tokens)
    response = response[0]["generated_text"].split("Ответ:")[-1].strip() # Оставляем только ответ модели
    
    return response

In [7]:
def final_pipeline(query: str, top_k: int = 3, debug: bool = False) -> str:
    """ 
    Пайплайн работы модели 

    :param query: Вопрос пользователя
    :param top_k: Количество возвращаемых документов добавленных в контекст модели
    :param debug: Флаг для вывода вопроса с подаваемым контекстом
    :return: Ответ модели
    """
    
    context = retrieve_documents(query, top_k) # фильтруем по городам и странам документы, затем оставляем top-k наиболее релевантных запросу

    if debug:
        print(f"Вопрос пользователя:\n{query}")
        print("\n" + "*" * 100 + "\n")
        print(f"Найденный контекст:\n{context}")
        print("\n" + "*" * 100 + "\n")
    answer = generate_answer(query, context, max_new_tokens=300) # генерируем ответ с учетом найденных документов
    
    return answer

# Метрика оценки генерации


Будем оценивать соответствие сгенерированного текста эталонному для набора из 10 вопросов. 

Оценка производится с помощью косинусной близости на эмбеддингах полученных от внешней модели FRIDA

Таким образом, для каждой тройки (вопрос | сгенерированный нашей LLM ответ| эталонный ответ составленный вручную) - у нас будет оценка от 0 до 1. В конце просто усредним оценки по всем парам

In [8]:
from sentence_transformers import SentenceTransformer
import torch

# Загрузим эмбеддер
embedder = SentenceTransformer("ai-forever/FRIDA")

In [44]:
from tqdm import tqdm
from typing import List, Dict

def evaluate_similarity_frida(
    questions: List[str], 
    generated_answers: List[str],
    reference_answers: List[str]
) -> List[float]:
    """
    Вычисляет косинусную близость между сгенерированными и эталонными ответами
    с помощью FRIDA SentenceTransformer.

    :param questions: Список вопросов
    :param generated_answers: Список ответов, сгенерированных LLM.
    :param reference_answers: Список эталонных ответов.
        
    :return: Косинусная близость по каждой паре.
    """

    inputs = (
        [f"search_document: {text}" for text in generated_answers] +
        [f"search_document: {text}" for text in reference_answers]
    )

    embeddings = embedder.encode(inputs, convert_to_tensor=True)

    gen_embs = embeddings[:len(generated_answers)]
    ref_embs = embeddings[len(generated_answers):]

    sim_scores = (gen_embs @ ref_embs.T).diagonal().tolist()
    print(len(sim_scores))
    for i, (question, generated, reference) in enumerate(zip(questions, generated_answers, reference_answers)):
        
        print(f"Вопрос: {question}:")
        #print(f"Ожидалось:\n{reference}")
        #print(f"Предсказано:\n{generated}")
        print(f"cosine_similarity: {sim_scores[i]:.4f}\n")
        print(100 * '*')

    avg_sim = sum(sim_scores) / len(sim_scores)
    print("\nОбщая оценка")
    print(f"Средний similarity: {avg_sim:.4f}")
    
    return sim_scores

In [23]:
import os

# считываем все вопросы и референсные ответы
base_dir = "questions"

questions = []
reference_answers = []

for country in os.listdir(base_dir):
    country_path = os.path.join(base_dir, country)

    if not os.path.isdir(country_path):
        continue

    for file in os.listdir(country_path):
        if file.endswith(".json"):
            file_path = os.path.join(country_path, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            for item in data:
                q = item.get("question", "").strip()
                a = item.get("answer", "").strip()

                questions.append(q)
                reference_answers.append(a)
                    

In [24]:
generated_answer = []

for question in tqdm(questions):
    response = final_pipeline(question) 
    generated_answer.append(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset 66.20s/it]
  6%|█████▏                                                                           | 6/93 [06:43<1:37:34, 67.29s/it]


KeyboardInterrupt: 

In [33]:
reference_answers

['Длина Берлинской стены составляла около 155 км, из которых 43,1 км приходились непосредственно на город, а высота достигала 4,1 м.',
 'В Берлине находятся такие музеи, как Музей Боде, Пергамский музей, Старый музей, Новый музей и Старая национальная галерея.',
 'В Берлине можно посетить оперу, мюзиклы, концерты, балет и театр.',
 'Некоторые оперные театры в Берлине включают Staatsoper Unter den Linden, Komische Oper и Deutsche Oper.',
 'Мюзиклы можно посмотреть в Friedrichstadt-Palast, Theater des Westens, Bluemax Theater, Admiralspalast и Wintergarten.',
 'Концерты проходят в Philharmonie, Kammermusiksaal, Konzerthaus и Kammermusiksaal Friedenau.']

In [45]:
evaluate_similarity_frida(questions, generated_answer, reference_answers)

6
Вопрос: Какова была длина и высота Берлинской стены?:
cosine_similarity: 0.2486

****************************************************************************************************
Вопрос: Какие музеи находятся в Берлине?:
cosine_similarity: 0.5772

****************************************************************************************************
Вопрос: Какие виды мероприятий можно посетить в Берлине?:
cosine_similarity: 0.8087

****************************************************************************************************
Вопрос: Какие оперные театры есть в Берлине?:
cosine_similarity: 0.7688

****************************************************************************************************
Вопрос: Где можно посмотреть мюзиклы в Берлине?:
cosine_similarity: 0.6104

****************************************************************************************************
Вопрос: Где проходят концерты в Берлине?:
cosine_similarity: 0.4874

****************************************

[0.24862411618232727,
 0.5771656632423401,
 0.8087441325187683,
 0.7687965035438538,
 0.610448956489563,
 0.4873986542224884]